In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

import os.path
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, Reshape, ReLU
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, Dropout, Flatten,Input
import time
from keras.utils.vis_utils import plot_model
from keras.datasets.fashion_mnist import load_data
from IPython import display

import glob
import random
import imageio
import PIL, cv2
import pandas as pd
%matplotlib inline
from skimage.morphology import convex_hull_image, erosion
from skimage.morphology import square
import matplotlib.image as mpimg
import skimage
import math
from scipy.ndimage.filters import convolve
from PIL import Image,ImageFilter
from skimage.feature import hessian_matrix, hessian_matrix_eigvals

<ipython-input-1-8b863c13f0ab>:26: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import convolve


#Generated Dataset

In [2]:
# create the folder to save the images

'''
folder_path = '/content/generated_fingerprints'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for i in range(6000):
    # Add some random noise to the input noise vector
    noise = tf.random.normal([1, noise_dim])
    
    # Generate a synthetic fingerprint using the generator function
    fingerprint = generator(noise, training=False)
    
    # Convert the TensorFlow tensor to a NumPy array
    fingerprint_np = fingerprint.numpy()
    
    # Convert the fingerprint array to a PIL Image object
    image = Image.fromarray((fingerprint_np[0] * 255).astype(np.uint8))
    
    # Save the image as a JPEG file in the folder
    filename = f'generated_fingerprint_{i}.jpg'
    file_path = os.path.join(folder_path, filename)
    image.save(file_path, 'JPEG')
'''

"\nfolder_path = '/content/generated_fingerprints'\nif not os.path.exists(folder_path):\n    os.makedirs(folder_path)\n\nfor i in range(6000):\n    # Add some random noise to the input noise vector\n    noise = tf.random.normal([1, noise_dim])\n    \n    # Generate a synthetic fingerprint using the generator function\n    fingerprint = generator(noise, training=False)\n    \n    # Convert the TensorFlow tensor to a NumPy array\n    fingerprint_np = fingerprint.numpy()\n    \n    # Convert the fingerprint array to a PIL Image object\n    image = Image.fromarray((fingerprint_np[0] * 255).astype(np.uint8))\n    \n    # Save the image as a JPEG file in the folder\n    filename = f'generated_fingerprint_{i}.jpg'\n    file_path = os.path.join(folder_path, filename)\n    image.save(file_path, 'JPEG')\n"

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil

#folder_path = '/content/generated_fingerprints'
#drive_path = '/content/gdrive/MyDrive/generated_fingerprints'
#os.makedirs(drive_path, exist_ok=True)
#for file_name in os.listdir(folder_path):
#    shutil.copy(os.path.join(folder_path, file_name), os.path.join(drive_path, file_name))

orig_folder = '/content/socofing/SOCOFing/Real'
gen_folder = '/content/generated_fingerprints'

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16

# Define paths to original and generated image folders
orig_folder = '/content/gdrive/MyDrive/Socofing_png'
gen_folder = '/content/gdrive/MyDrive/generated_fingerprints'

# Initialize VGG16 model and get reference to the last convolutional layer
vgg_model = VGG16(weights='imagenet', include_top=False)
last_layer = vgg_model.get_layer('block5_conv3')

# Create a new model that outputs the last convolutional layer's feature map
feature_extractor = tf.keras.models.Model(inputs=vgg_model.input, outputs=last_layer.output)

from PIL import Image

def preprocess(img_path):
    img = Image.open(img_path)
    img = img.convert('RGB')  # convert to RGB format if necessary
    img = img.resize((224, 224))  # resize the image
    x = np.asarray(img)  # convert to numpy array
    x = np.expand_dims(x, axis=0)  # add batch dimension
    x = preprocess_input(x)  # preprocess using VGG16 preprocessing
    return x

# Extract features for original images
orig_features = []
for filename in os.listdir(orig_folder):
    if filename.endswith('.png'):
        img_path = os.path.join(orig_folder, filename)
        x = preprocess(img_path)
        features = feature_extractor.predict(x)
        features = np.reshape(features, (features.shape[0], -1))
        orig_features.append(features)

# Extract features for generated images
gen_features = []
for filename in os.listdir(gen_folder):
    if filename.endswith('.jpg'):
        img_path = os.path.join(gen_folder, filename)
        x = preprocess(img_path)
        features = feature_extractor.predict(x)
        features = np.reshape(features, (features.shape[0], -1))
        gen_features.append(features)

In [8]:
from sklearn.neighbors import NearestNeighbors

orig_features = np.concatenate(orig_features, axis=0)
gen_features = np.concatenate(gen_features, axis=0)

# Concatenate original and generated feature vectors into a single array
all_features = np.concatenate([orig_features, gen_features], axis=0)

# Fit a KNN model on the original feature vectors
n_neighbors = 5  # Choose the number of neighbors for the KNN model
knn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
knn_model.fit(orig_features)

# Compute the average distance from each generated feature vector to its nearest neighbor in the original dataset
distances, _ = knn_model.kneighbors(gen_features)
avg_dist = np.mean(distances)

print("KNN score: ", avg_dist)


KNN score:  1479.2690989948542


Quando avevo fatto tutte e 6000 le immagine (BMP) lo score era circa a 1555. Con le prime 600 convertite (png) fa 1479 (e ci mette 18 minuti).